In [1]:
import numpy as np
from skimage import io, data
from skimage.color import rgb2lab, lab2rgb
from sklearn.neighbors import KNeighborsRegressor
import os

In [2]:
data_dir = "./data/knn/knn_data/"
input_path = "./data/knn/input.jpg"
output_path = "knn_output.jpg"
block_size = 1

In [3]:
# 读入风格图像, 得到X, Y
# X: 3*3像素格的灰度值
# Y: 中心像素格的色彩值
def read_style_image(file_name, size=block_size):
    img = rgb2lab(io.imread(file_name))
    w, h = img.shape[:2]
    X = []
    Y = []
    for x in range(size, w - size):
        for y in range(size, h - size):
            X.append(img[x - size : x + size + 1, y - size : y + size + 1, 0].reshape(-1))
            Y.append(img[x, y, 1:])
    return X, Y

In [4]:
# 根据若干张彩色图像构造数据集
# X: 3*3像素格的灰度值
# Y: 中心像素格的色彩值
def create_dataset(data_dir=data_dir, size=1):
    X = []
    Y = []
    n = 0
    for file in os.listdir(data_dir):
        print("reading", file)
        X0, Y0 = read_style_image(os.path.join(data_dir, file))
        X.extend(X0)
        Y.extend(Y0)
        n += 1
        if n >= size:
            break
    return X, Y

In [5]:
print("reading data")
X, Y = create_dataset()
print("finish reading")

reading data
reading kNNStyle.jpg
finish reading


In [6]:
# 构造KNN模型，设置K=4, 预测的色彩值是与其最近的4个点的色彩值的加权平均
# 权重与距离的倒数成正比
nbrs = KNeighborsRegressor(n_neighbors=4, weights='distance')

print("start fitting")
nbrs.fit(X, Y)
print("finish fitting")

start fitting
finish fitting


In [7]:
# 处理原图像得到X
def split_origin_image(img, size=block_size):
    w, h = img.shape[:2]
    X = []
    for x in range(size, w - size):
        for y in range(size, h - size):
            X.append(img[x - size : x + size + 1, y - size : y + size + 1, 0].reshape(-1))
    return X

In [8]:
# 根据KNN算法得到风格迁移后的图像
def rebuild(file_name, size=block_size):
    img = rgb2lab(io.imread(file_name))
    w, h = img.shape[:2]
    photo = np.zeros([w, h, 3])
    X = split_origin_image(img)
    print("start predicting")
    p_ab = nbrs.predict(X).reshape(w - 2 * size, h - 2 * size, -1)
    print("finish predicting")
    for x in range(size, w - size):
        for y in range(size, h - size):
            photo[x, y, 0] = img[x, y, 0]
            photo[x, y, 1] = p_ab[x - size, y - size, 0]
            photo[x, y, 2] = p_ab[x - size, y - size, 1]
    return photo

In [9]:
new_photo = rebuild(input_path)

start predicting
finish predicting


In [10]:
# 显示输出图像
from IPython.display import display
display(lab2rgb(new_photo))

array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.98276003, 0.93693233, 0.67400384],
        [0.97783824, 0.93319112, 0.67186206],
        ...,
        [0.57922611, 0.74639528, 0.77978305],
        [0.64679917, 0.74701502, 0.72380214],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.98726262, 0.93587738, 0.66822929],
        [0.97303181, 0.93548908, 0.65974577],
        ...,
        [0.8193591 , 0.71886107, 0.4024799 ],
        [0.73340663, 0.74457097, 0.6067231 ],
        [0.        , 0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.35363203, 0.57944573, 0.71119451],
        [0.34335161, 0